# Food-101: Entrenamiento del Modelo

**Objetivo:** Entrenar un modelo de clasificación de 101 tipos de alimentos usando Transfer Learning con MobileNetV2.

In [1]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [2]:
# Parametros basicos
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5  # Empezar con pocas epocas
AUTOTUNE = tf.data.AUTOTUNE

## 1. Cargar datos y crear pipeline

In [3]:
# Cargar Food-101 dataset
(train_ds, val_ds), info = tfds.load(
    'food101',
    split=['train', 'validation'],
    with_info=True,
    as_supervised=True
)

class_names = info.features['label'].names
num_classes = len(class_names)
print(f'Clases: {num_classes}')
print(f'Training samples: {info.splits["train"].num_examples}')
print(f'Validation samples: {info.splits["validation"].num_examples}')

Clases: 101
Training samples: 75750
Validation samples: 25250


In [4]:
# Funciones de preprocesamiento y augmentacion
def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image, label

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label

In [5]:
# Crear pipelines de datos
train_dataset = (
    train_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

val_dataset = (
    val_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

print('Pipelines creados')

Pipelines creados


In [6]:
# Verificar un batch
for images, labels in train_dataset.take(1):
    print(f'Batch de imagenes: {images.shape}')
    print(f'Batch de labels: {labels.shape}')
    print(f'Rango de valores: [{tf.reduce_min(images):.2f}, {tf.reduce_max(images):.2f}]')

Batch de imagenes: (32, 224, 224, 3)
Batch de labels: (32,)
Rango de valores: [0.00, 1.00]


## 2. Crear modelo (Transfer Learning con MobileNetV2)

In [7]:
# Modelo base: MobileNetV2 preentrenado en ImageNet
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Congelar el modelo base para transfer learning
base_model.trainable = False

print('Modelo base cargado: MobileNetV2')
print(f'Pesos congelados: {not base_model.trainable}')

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Modelo base cargado: MobileNetV2
Pesos congelados: True


In [8]:
# Construir modelo completo
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 101)            │       129,381 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## 3. Compilar y entrenar

In [ ]:
# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('Modelo compilado con adam optimizer')

Modelo compilado con adam optimizer


: 

In [ ]:
# Entrenar el modelo
print(f'Entrenando por {EPOCHS} epocas...')

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

Entrenando por 5 epocas...
Epoch 1/5
1738/2368 ━━━━━━━━━━━━━━━━━━━━ 3:17 314ms/step - accuracy: 0.3145 - loss: 2.9605

## 4. Visualizar entrenamiento

In [ ]:
# Graficar accuracy y loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax1.plot(history.history['accuracy'], label='Train')
ax1.plot(history.history['val_accuracy'], label='Validation')
ax1.set_title('Accuracy por Epoca')
ax1.set_xlabel('Epoca')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(history.history['loss'], label='Train')
ax2.plot(history.history['val_loss'], label='Validation')
ax2.set_title('Loss por Epoca')
ax2.set_xlabel('Epoca')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('results/training_history.png')
plt.show()

## 5. Evaluar modelo en validacion

In [ ]:
# Evaluar en el conjunto de validacion
test_loss, test_acc = model.evaluate(val_dataset)
print(f'\nValidation Loss: {test_loss:.4f}')
print(f'Validation Accuracy: {test_acc:.4f}')
print(f'\nMejora sobre random (1/{num_classes}): {test_acc / (1/num_classes):.1f}x')

In [ ]:
# Obtener predicciones en validacion
print('Generando predicciones...')

y_true = []
y_pred = []

for images, labels in val_dataset:
    predictions = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(f'Predicciones generadas: {len(y_pred)}')

## 6. Analizar resultados (Confusion Matrix)

In [ ]:
# Matriz de confusion para las primeras 20 clases (para visualizacion)
top_n = 20
mask = (y_true < top_n) & (y_pred < top_n)
y_true_top = y_true[mask]
y_pred_top = y_pred[mask]

cm = confusion_matrix(y_true_top, y_pred_top, labels=range(top_n))

# Visualizar
plt.figure(figsize=(12, 10))
plt.imshow(cm, interpolation='nearest', cmap='Blues')
plt.title(f'Confusion Matrix (Top {top_n} clases)')
plt.colorbar()
tick_marks = np.arange(top_n)
plt.xticks(tick_marks, [class_names[i][:15] for i in range(top_n)], rotation=90)
plt.yticks(tick_marks, [class_names[i][:15] for i in range(top_n)])
plt.ylabel('Verdadero')
plt.xlabel('Predicho')
plt.tight_layout()
plt.savefig('results/confusion_matrix_top20.png')
plt.show()

print(f'Diagonal (predicciones correctas): {np.diag(cm).sum()} / {cm.sum()}')

In [ ]:
# Classification report
report = classification_report(
    y_true, 
    y_pred, 
    target_names=class_names,
    digits=3
)

print('\nClassification Report:\n')
print(report)

# Guardar reporte
with open('results/classification_report.txt', 'w') as f:
    f.write(report)

## 7. Visualizar predicciones

In [ ]:
# Obtener algunas imagenes con sus predicciones
sample_images = []
sample_labels = []
sample_predictions = []

for images, labels in val_dataset.take(3):
    predictions = model.predict(images, verbose=0)
    sample_images.extend(images.numpy())
    sample_labels.extend(labels.numpy())
    sample_predictions.extend(np.argmax(predictions, axis=1))

# Encontrar predicciones correctas e incorrectas
correct_indices = [i for i in range(len(sample_labels)) if sample_labels[i] == sample_predictions[i]]
incorrect_indices = [i for i in range(len(sample_labels)) if sample_labels[i] != sample_predictions[i]]

In [ ]:
# Visualizar predicciones correctas
plt.figure(figsize=(12, 8))
for i in range(min(9, len(correct_indices))):
    idx = correct_indices[i]
    plt.subplot(3, 3, i + 1)
    plt.imshow(sample_images[idx])
    plt.title(f'Correcto: {class_names[sample_labels[idx]][:15]}')
    plt.axis('off')
plt.suptitle('Predicciones Correctas', fontsize=16)
plt.tight_layout()
plt.savefig('results/correct_predictions.png')
plt.show()

In [ ]:
# Visualizar predicciones incorrectas
if len(incorrect_indices) > 0:
    plt.figure(figsize=(12, 8))
    for i in range(min(9, len(incorrect_indices))):
        idx = incorrect_indices[i]
        plt.subplot(3, 3, i + 1)
        plt.imshow(sample_images[idx])
        plt.title(f'Real: {class_names[sample_labels[idx]][:12]}\nPred: {class_names[sample_predictions[idx]][:12]}', 
                  fontsize=9)
        plt.axis('off')
    plt.suptitle('Predicciones Incorrectas', fontsize=16)
    plt.tight_layout()
    plt.savefig('results/incorrect_predictions.png')
    plt.show()
else:
    print('No hay predicciones incorrectas en la muestra')

## 8. Guardar todo (modelo + resultados)

In [ ]:
# Guardar el modelo
model.save('models/food_model_v1.h5')
print('Modelo guardado en: models/food_model_v1.h5')

In [ ]:
# Guardar historial de entrenamiento
history_df = pd.DataFrame(history.history)
history_df.to_csv('results/training_history.csv', index=False)
print('Historial guardado en: results/training_history.csv')

In [ ]:
# Guardar matriz de confusion completa
cm_full = confusion_matrix(y_true, y_pred, labels=range(num_classes))
cm_df = pd.DataFrame(cm_full, index=class_names, columns=class_names)
cm_df.to_csv('results/confusion_matrix_full.csv')
print('Confusion matrix guardada en: results/confusion_matrix_full.csv')

## 9. Resumen final

In [ ]:
print('='*60)
print('RESUMEN DEL ENTRENAMIENTO')
print('='*60)
print(f'Modelo: MobileNetV2 + Transfer Learning')
print(f'Dataset: Food-101 ({num_classes} clases)')
print(f'Epocas: {EPOCHS}')
print(f'Batch size: {BATCH_SIZE}')
print(f'\nResultados:')
print(f'  Training Accuracy:   {history.history["accuracy"][-1]:.4f}')
print(f'  Validation Accuracy: {test_acc:.4f}')
print(f'  Training Loss:       {history.history["loss"][-1]:.4f}')
print(f'  Validation Loss:     {test_loss:.4f}')
print(f'\nMejora sobre random: {test_acc / (1/num_classes):.1f}x')
print('='*60)
print('\nProximos pasos:')
print('  1. Entrenar mas epocas (10-15)')
print('  2. Fine-tuning (descongelar capas del base_model)')
print('  3. Probar modelos mas grandes (EfficientNet)')
print('  4. Analizar clases con peor performance')
print('='*60)